__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x36_d02_identifizieren.ipynb)__

# Identifizieren
* speichert in data_02
* lex_test level 2

## Import

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 17:51:44


time: 409 ms


In [2]:
# alle anderen
try:
    import pandasklar as pak, bpyth as bpy, bj_nlp, swifter

except ImportError as e:
    !pip install pandasklar swifter bj_nlp
    import import pandasklar as pak, bpyth as bpy, bj_nlp, swifter
    
grid       = pak.grid
check_mask = pak.check_mask      

time: 4.2 s


## Einstellungen

In [5]:
# Einstellungen

# Was debuggen?
suche_debug = ['umhänge','umhängst','umhängen']  
suche_debug = ['inner-','innere','inner']  
suche_debug = ['Citrone','zuviele']  
suche_debug      = ['gefriergetrocknet','gefriertrocknen','missraten']  

gründlich = True # einige Arbeitsschritte können weggelassen werden, wenn gründlich = True
version = '_all'
#version = '_100K'

# verbose
pak.Config.set('VERBOSE', True)

# Load
verzeichnis_load                         = 'data_01'
wiktionary_lemma_filename                = verzeichnis_load + '/wiktionary_lemma'        + version + '.pickle'  
wiktionary_nolemma_filename              = verzeichnis_load + '/wiktionary_nolemma'      + version + '.pickle'  
wiktionary_merkmal_filename              = verzeichnis_load + '/wiktionary_merkmal'      + version + '.pickle'  
wiktionary_merkmal_text_filename         = verzeichnis_load + '/wiktionary_merkmal_text' + version + '.pickle'  

# Steuertabellen
lex_test_filename                        = 'Steuertabellen/lex_test.xlsx'  
löschliste_filename                      = 'Steuertabellen/löschliste.xlsx'     
     
# Save
verzeichnis_save                         = 'data_02'
wiktionary_lemma_filename_save           = verzeichnis_save + '/wiktionary_lemma.pickle'  
wiktionary_nolemma_filename_save         = verzeichnis_save + '/wiktionary_nolemma.pickle'  # wird als translate_lex wieder geladen
wiktionary_merkmal_filename_save         = verzeichnis_save + '/wiktionary_merkmal.pickle'  
wiktionary_merkmal_text_filename_save    = verzeichnis_save + '/wiktionary_merkmal_text.pickle'  
wiktionary_merkmal_wertlos_filename_save = verzeichnis_save + '/wiktionary_merkmal_wertlos.pickle'  


VERBOSE = True
--> setting verbose=True as default for all pandasklar functions

time: 48.9 ms


## Vorbereiten

In [4]:
# Verzeichnisse erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 
os.makedirs(verzeichnis_save, exist_ok=True) 

time: 44.4 ms


In [5]:
# Steuertabellen und Python-Files von Github herunterladen
import os
if not os.path.exists('Steuertabellen'):
   !git clone https://github.com/djekra/wiktionary_wortschatz.git
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/Steuertabellen Steuertabellen
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/*.py .
#   !rm -r wiktionary_wortschatz

time: 38.9 ms


In [6]:
# Lade Ergebnisse der vorherigen Verarbeitungsstufe von Google Drive, falls nicht lokal vorhanden
# Dieses Verzeichnis wurde in der vorherigen Verarbeitungsstufe angelegt 
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av /content/drive/MyDrive/{verzeichnis_load} .

time: 43.8 ms


In [3]:
# Weitere Imports
from Steuertabellen.s01_Steuertabellen import *
from x32_Funktionen                    import *

time: 1.56 s


In [6]:
#!python -m spacy download de_core_news_sm

time: 44.4 ms


In [7]:
# nlp-Objekt erstellen
   
if not 'blp' in globals() or True:
    blp = bj_nlp.blpClass(lang='de', verbose=False)    
    
nlp     = blp.nlp    
nlp.blp = blp

# Pipeline
nlp.remove_pipe('parser')
blp.analyze_pipes()

Input rtype=('dict', 'dict', 'list', 'str') shape=(-77, -77)
rotated=True Output rtype=('DataFrame', 'Series') shape=(4, 6)
model_name: de_core_news_sm
vocab_len: 682
pipe component disabled: ['senter']


,component,assigns,requires,scores,retokenizes
0,tok2vec,[doc.tensor],[],[],False
1,tagger,[token.tag],[],[tag_acc],False
2,morphologizer,"[token.morph, token.pos]",[],"[pos_acc, morph_acc, morph_per_feat]",False
3,lemmatizer,[token.lemma],[],[lemma_acc],False
4,attribute_ruler,[],[],[],False
5,ner,"[doc.ents, token.ent_iob, token.ent_type]",[],"[ents_f, ents_p, ents_r, ents_per_type]",False


time: 1.18 s


In [8]:
# Ausprobieren
text = 'Hallo {{NNBSP}} Welt'
doc = nlp(text)
doc[1] # }

{

time: 63.8 ms


## Daten laden

In [9]:
# Laden     
wiktionary_lemma         = pak.load_pickle(wiktionary_lemma_filename)
wiktionary_nolemma       = pak.load_pickle(wiktionary_nolemma_filename)
wiktionary_merkmal       = pak.load_pickle(wiktionary_merkmal_filename)
wiktionary_merkmal_text  = pak.load_pickle(wiktionary_merkmal_text_filename)

168017 rows loaded
644849 rows loaded
4987597 rows loaded
621260 rows loaded
time: 6.93 s


In [10]:
# lex_test, lemma_test
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=2 )
grid(lemma_test)

74 rows loaded
Delete 17 rows from 74
47 rows


time: 1.89 s


In [11]:
# plan_merkmal Steuertabelle
plan_merkmal = plan_merkmal_erstellen()
grid(plan_merkmal, rows=9, column_definitions={ 'collect': {'maxWidth': 50,}, ' ': {'maxWidth': 50,}                   },     )

103 rows


time: 211 ms


In [12]:
#mask = wiktionary_lemma.lemma == 'gefriergetrocknet'
#wiktionary_lemma[mask]
search_str(wiktionary_lemma, suche_debug)

,page_id,page_part,section_id2,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,score,data_id
129338,-1,-1,gefriergetrocknet,gefriergetrocknet,gefriergetrocknet,False,1,ADJA,ADJA,ATraUFcN5L,ADJ,A,Ptz2 noSteig manuell,0.500000,gefriertrocknen
129339,273380,0,gefriergetrocknet,gefriergetrocknet,gefriergetrocknet,False,1,ADJA,ADJA,M9yT3byKKu,ADJ,A,noSteig,3.807552,
129340,1054310,0,gefriertrocknen,gefriertrocknen,gefriertrocknen,False,1,VVFIN,VVFIN,TLhfFhOWFO,VER,V,,14.422655,
129341,-1,-1,gefriertrocknen,gefriertrocknen,gefriertrocknen,False,1,VVFIN,VVFIN,dFIK2wex3d,VER,V,manuell,0.450000,
129342,1054319,0,gefriertrocknend,gefriertrocknend,gefriertrocknend,False,1,ADJA,ADJA,Ah4pELiP74,ADJ,A,Ptz1,0.467347,gefriertrocknen
142186,242150,0,missraten,missraten,missraten,False,1,VVFIN,VVFIN,foLRX8vFnm,VER,V,,4.472594,
142187,-1,-1,missraten,missraten,missraten,False,1,VVFIN,VVFIN,hMbvB8jNwL,VER,V,manuell,0.450000,
142188,-1,-1,missraten_MADJ,missraten,missraten,False,1,ADJA,ADJA,IBpowZE9hp,ADJ,A,Ptz2 manuell,0.500000,missraten
142189,446280,0,missratend,missratend,missratend,False,1,ADJA,ADJA,i9VVIKLe4m,ADJ,A,Ptz1,0.579592,missraten
142562,-1,-1,mißraten_MADJ,mißraten,mißraten,False,1,ADJA,ADJA,rNQ5memXrO,ADJ,A,Ptz2 manuell,0.500000,missraten


time: 318 ms


In [13]:
# suche_debug wiktionary_merkmal
search_str(wiktionary_merkmal, suche_debug)

,section_id2,merkmal,data,num,meta,data_id,score
252288,raten,familie,missraten,,Adjektiv,,16.867235
255828,gefrieren,familie,gefriertrocknen,,Adjektive,,13.330085
264234,fehlschlagen,syn,missraten,1,,,6.317905
515080,ausrutschen,lateral,missraten,3,,,9.354445
519887,Trockenblume,syn,gefriergetrocknet,1,,,7.348356
...,...,...,...,...,...,...,...
4097356,missraten,ipa,mɪsˈʁaːtn,<NA>,,,4.472594
4102697,gefriergetrocknet,ipa,ɡəˈfʁiːɐɡəˌtʁɔknət,<NA>,,,3.807552
4227770,missraten,syllables,miss⇀ra⇀ten,<NA>,,,4.472594
4342447,gefriergetrocknet,syllables,ge⇀frier⇀ge⇀trock⇀net,<NA>,,,3.807552


time: 6.28 s


In [14]:
# suche_debug wiktionary_nolemma
search_str(wiktionary_nolemma, suche_debug)

,nolemma_id,lex,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,score
63720,gefriergetrocknetem,gefriergetrocknetem,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,qfxGEFcaEM,<NA>,,Deklinierte_Form,0.822449
63721,gefriergetrockneten,gefriergetrockneten,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,cSrtFUpYCE,<NA>,,Deklinierte_Form,2.253350
63722,gefriergetrockneter,gefriergetrockneter,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,cJeBRmnkEZ,<NA>,,Deklinierte_Form,1.298417
63723,gefriergetrocknetes,gefriergetrocknetes,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,ztSD6oEkot,<NA>,,Deklinierte_Form,1.166870
63724,gefriergetrocknete,gefriergetrocknete,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,UYh5zwEhfx,<NA>,,Deklinierte_Form,1.848965
178159,missrietet,missrietet,missraten,missraten,LEX_V,LEX_V,y9ZTeYS8SU,<NA>,,Konjugierte_Form Verb,0.963265
178160,missrieten,missrieten,missraten,missraten,LEX_V,LEX_V,VhpO1YDYiF,<NA>,,Konjugierte_Form Verb,1.144945
178161,missrietst,missrietst,missraten,missraten,LEX_V,LEX_V,5Jms1aP5H9,<NA>,,Konjugierte_Form Verb,0.812245
178162,missrietest,missrietest,missraten,missraten,LEX_V,LEX_V,N0lyUCwTOa,<NA>,,Konjugierte_Form Verb,0.993878
178163,missriete,missriete,missraten,missraten,LEX_V,LEX_V,p6ksoWbv4K,<NA>,,Konjugierte_Form Verb,0.810204


time: 651 ms


In [15]:
#pak.analyse_cols(wiktionary_merkmal_text)

time: 32.7 ms


## IDs schreiben, Erstbearbeitung, Daten übertragen

### wiktionary_merkmal

In [16]:
# is_lex nach wiktionary_merkmal übertragen
#wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,'is_lex')
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, plan_merkmal, on='merkmal', col='is_lex', func='min') 
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal, 'is_lex', 'merkmal')

update_col: func min applied, 3 records less!
update_col: 4987597 cells written into new column
time: 11.5 s


In [17]:
# is_lex manuell
mask1 = wiktionary_merkmal.is_lex.isnull()
mask2 = wiktionary_merkmal.meta.str.contains('manuell')
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0,5)
wiktionary_merkmal.loc[mask,'is_lex'] = True
wiktionary_merkmal[mask]

check_mask:  0 rows


,section_id2,merkmal,is_lex,data,num,meta,data_id,score


time: 1.37 s


### wiktionary_lemma

In [18]:
# lemma_id 
wiktionary_lemma   = pak.rename_col( wiktionary_lemma,   'section_id2', 'lemma_id' )
wiktionary_merkmal = pak.rename_col( wiktionary_merkmal, 'section_id2', 'lemma_id' )

time: 1.22 s


In [19]:
# suche_debug wiktionary_lemma
search_str(wiktionary_lemma, suche_debug)

,page_id,page_part,lemma_id,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,score,data_id
129338,-1,-1,gefriergetrocknet,gefriergetrocknet,gefriergetrocknet,False,1,ADJA,ADJA,ATraUFcN5L,ADJ,A,Ptz2 noSteig manuell,0.500000,gefriertrocknen
129339,273380,0,gefriergetrocknet,gefriergetrocknet,gefriergetrocknet,False,1,ADJA,ADJA,M9yT3byKKu,ADJ,A,noSteig,3.807552,
129340,1054310,0,gefriertrocknen,gefriertrocknen,gefriertrocknen,False,1,VVFIN,VVFIN,TLhfFhOWFO,VER,V,,14.422655,
129341,-1,-1,gefriertrocknen,gefriertrocknen,gefriertrocknen,False,1,VVFIN,VVFIN,dFIK2wex3d,VER,V,manuell,0.450000,
129342,1054319,0,gefriertrocknend,gefriertrocknend,gefriertrocknend,False,1,ADJA,ADJA,Ah4pELiP74,ADJ,A,Ptz1,0.467347,gefriertrocknen
142186,242150,0,missraten,missraten,missraten,False,1,VVFIN,VVFIN,foLRX8vFnm,VER,V,,4.472594,
142187,-1,-1,missraten,missraten,missraten,False,1,VVFIN,VVFIN,hMbvB8jNwL,VER,V,manuell,0.450000,
142188,-1,-1,missraten_MADJ,missraten,missraten,False,1,ADJA,ADJA,IBpowZE9hp,ADJ,A,Ptz2 manuell,0.500000,missraten
142189,446280,0,missratend,missratend,missratend,False,1,ADJA,ADJA,i9VVIKLe4m,ADJ,A,Ptz1,0.579592,missraten
142562,-1,-1,mißraten_MADJ,mißraten,mißraten,False,1,ADJA,ADJA,rNQ5memXrO,ADJ,A,Ptz2 manuell,0.500000,missraten


time: 209 ms


In [20]:
# fillna
wiktionary_lemma['data_id'] = wiktionary_lemma.data_id.fillna('')

time: 51.3 ms


In [21]:
# group + agg_words_nodup

cols_vorher = wiktionary_lemma.columns

wiktionary_lemma = pak.group_and_agg( wiktionary_lemma, 
              ['lemma_id',  'page_id',  'page_part',  'lemma',  'tag',                 'member',             'data_id', 'score',  ],
              [ 'group',    'max',      'max',        'first',  pak.agg_words_nodup,   pak.agg_words_nodup,  'max',     'sum',    ],
              ['lemma_id',  'page_id',  'page_part',  'lemma',  'tag',                 'member',             'data_id', 'score',  ],
              optimize=True
             )

162679 unique rows and 5338 rows with duplicates
2671 rows less, now 165346 rows
time: 3.44 s


In [22]:
# suche_debug wiktionary_lemma
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,tag,member,data_id,score
2054,gefriergetrocknet,273380,0,gefriergetrocknet,ADJA,Ptz2 noSteig manuell,gefriertrocknen,4.307552
2055,gefriertrocknen,1054310,0,gefriertrocknen,VVFIN,manuell,,14.872655
2194,missraten,242150,0,missraten,VVFIN,manuell,,4.922594
127895,gefriertrocknend,1054319,0,gefriertrocknend,ADJA,Ptz1,gefriertrocknen,0.467347
140462,missraten_MADJ,-1,-1,missraten,ADJA,Ptz2 manuell,missraten,0.500000
140463,missratend,446280,0,missratend,ADJA,Ptz1,missraten,0.579592
140820,mißraten_MADJ,-1,-1,mißraten,ADJA,Ptz2 manuell,missraten,0.500000


time: 126 ms


In [23]:
# nach Priorität sortieren (laut Steuertabellen)
wiktionary_lemma['tag'] = tag_sortieren(wiktionary_lemma.tag)

time: 191 ms


In [24]:
# Spalten wiederherstellen
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1
wiktionary_lemma = ergänze_felder(wiktionary_lemma)                     # lemma_lower, isupper, lemma_len

# Fehler?
mask = (wiktionary_lemma.tagZZ.str.len() > 1)
check_mask(wiktionary_lemma,mask,0,10) # ein paar können wir tolerieren

check_mask:  287 rows
check_mask:  0 rows
time: 3.42 s


In [25]:
bpy.minivenn(cols_vorher, wiktionary_lemma.columns, format='diff')
assert cols_vorher == wiktionary_lemma.columns

time: 34.5 ms


In [26]:
# # TODO Klappt noch nicht # ADV ergänzen. Ebenso: Substantiv, istName, Name
mask1 =  wiktionary_lemma.member.str.contains('Substantiv')
mask2 = ~wiktionary_lemma.tag.str.contains('NN')
mask3 = ~wiktionary_lemma.tag.str.contains(' ') 
mask = mask1  &  mask2  &  mask3
grid(wiktionary_lemma, mask)


3 rows out of 165346


time: 242 ms


In [27]:
# AFIX MFIX ZFIX und zusätzliches TODO 
r = {'AFIX TRUNC':'AFIX', 
     'MFIX TRUNC':'MFIX', 
     'ZFIX TRUNC':'ZFIX', 
     ' TODO':'', 
    }
wiktionary_lemma['tag'] = pak.replace_str(wiktionary_lemma.tag, r)

time: 345 ms


In [28]:
# Spalten nach den Ersetzungen wiederherstellen
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1
wiktionary_lemma = ergänze_felder(wiktionary_lemma)                     # lemma_lower, isupper, lemma_len

# Fehler?
mask = (wiktionary_lemma.tagZZ.str.len() > 1)
check_mask(wiktionary_lemma, mask, 0) # diesmal tolerieren wir nichts. tagZZ ist jetzt eindeutig.

check_mask:  287 rows
check_mask:  0 rows
time: 4.09 s


In [29]:
# alle_ptz
mask_Ptz = wiktionary_lemma.data_id != ''
alle_ptz = wiktionary_lemma[mask_Ptz]
grid(alle_ptz,pak.sample)

16 rows out of 26944


time: 713 ms


In [30]:
# alle_ptz enthält nur Adjektive
mask = alle_ptz.tagZZ != 'A'
check_mask(alle_ptz,mask,0)
#alle_ptz[mask]

check_mask:  0 rows
time: 61.3 ms


In [31]:
# Alle alle_ptz.data_id weisen auf Verben
mask1 = wiktionary_lemma.lemma_id.isin(alle_ptz.data_id)
mask2 = wiktionary_lemma.tagZZ != 'V'
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask,0)
#wiktionary_lemma[mask]

check_mask:  0 rows
time: 129 ms


In [32]:
# suche_debug wiktionary_lemma
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score
2054,gefriergetrocknet,273380,0,gefriergetrocknet,gefriergetrocknet,False,1,ADJA,ADJA,s6ixiUi9BM,ADJ,A,Ptz2 noSteig manuell,gefriertrocknen,4.307552
2055,gefriertrocknen,1054310,0,gefriertrocknen,gefriertrocknen,False,1,VVFIN,VVFIN,yJnWihTkXL,VER,V,manuell,,14.872655
2194,missraten,242150,0,missraten,missraten,False,1,VVFIN,VVFIN,DXXh04S9JZ,VER,V,manuell,,4.922594
127895,gefriertrocknend,1054319,0,gefriertrocknend,gefriertrocknend,False,1,ADJA,ADJA,1HqGYAdhah,ADJ,A,Ptz1,gefriertrocknen,0.467347
140462,missraten_MADJ,-1,-1,missraten,missraten,False,1,ADJA,ADJA,dJ48jJBIod,ADJ,A,Ptz2 manuell,missraten,0.500000
140463,missratend,446280,0,missratend,missratend,False,1,ADJA,ADJA,xpAe81jVwT,ADJ,A,Ptz1,missraten,0.579592
140820,mißraten_MADJ,-1,-1,mißraten,mißraten,False,1,ADJA,ADJA,DpUQoGbzJB,ADJ,A,Ptz2 manuell,missraten,0.500000


time: 300 ms


### wiktionary_nolemma
* in wiktionary_nolemma sind keine Lemmas, sondern irgendwelche Lexeme, die aber einen eigene section auf der Wiktionary-Seite hatten
* nolemma_id ist der Schlüssel zu weiteren Merkmalen dieser Lexeme (siehe wiktionary_nomerkmal)
* lemma_id ist der Schlüssel zum Lemma (siehe wiktionary_lemma)

* Es wird versucht, jedes lemma in wiktionary_lemma zu finden und eine lemma_id zuzuweisen.
* Dazu wird zuallererst `lemma_tagZZ` geschrieben, also eine Grobklassifizierung des lemmas.

In [33]:
# suche_debug wiktionary_lemma
search_str(wiktionary_nolemma, suche_debug)

,nolemma_id,lex,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,score
63720,gefriergetrocknetem,gefriergetrocknetem,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,qfxGEFcaEM,<NA>,,Deklinierte_Form,0.822449
63721,gefriergetrockneten,gefriergetrockneten,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,cSrtFUpYCE,<NA>,,Deklinierte_Form,2.253350
63722,gefriergetrockneter,gefriergetrockneter,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,cJeBRmnkEZ,<NA>,,Deklinierte_Form,1.298417
63723,gefriergetrocknetes,gefriergetrocknetes,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,ztSD6oEkot,<NA>,,Deklinierte_Form,1.166870
63724,gefriergetrocknete,gefriergetrocknete,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,UYh5zwEhfx,<NA>,,Deklinierte_Form,1.848965
178159,missrietet,missrietet,missraten,missraten,LEX_V,LEX_V,y9ZTeYS8SU,<NA>,,Konjugierte_Form Verb,0.963265
178160,missrieten,missrieten,missraten,missraten,LEX_V,LEX_V,VhpO1YDYiF,<NA>,,Konjugierte_Form Verb,1.144945
178161,missrietst,missrietst,missraten,missraten,LEX_V,LEX_V,5Jms1aP5H9,<NA>,,Konjugierte_Form Verb,0.812245
178162,missrietest,missrietest,missraten,missraten,LEX_V,LEX_V,N0lyUCwTOa,<NA>,,Konjugierte_Form Verb,0.993878
178163,missriete,missriete,missraten,missraten,LEX_V,LEX_V,p6ksoWbv4K,<NA>,,Konjugierte_Form Verb,0.810204


time: 688 ms


In [34]:
# lex == lemma abtrennen
mask = wiktionary_nolemma.lex == wiktionary_nolemma.lemma
wiktionary_nolemma, df = pak.move_rows(wiktionary_nolemma,mask,msg=None)
#wiktionary_nolemma.loc[mask,'lemma'] = '' # TODO
df['lemma'] = ''

Move 54 rows from 644849
time: 879 ms


In [35]:
pak.sample(df)

,nolemma_id,lex,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,score
120,des,des,,des,LEX_A,LEX_A,VcEqMU7kd0,<NA>,,Deklinierte_Form Artikel,4.706456
2415,Wecken_2,Wecken,,wecken,LEX_A,LEX_A,4fN72Kodlj,<NA>,,Deklinierte_Form,0.895918
37296,zugrunde_liegen_1,zugrunde liegen,,zugrunde liegen,LEX_V,LEX_V,BDK74c4iDg,<NA>,,Konjugierte_Form Verb Wortverbindung,1.344702
131013,Thors_1,Thors,,thors,LEX_A,LEX_A,ebkM4tNjw0,<NA>,,Deklinierte_Form,0.055102
151298,übergösse_1,übergösse,,übergösse,LEX_V,LEX_V,x2Uj0lvBjq,<NA>,,Konjugierte_Form Verb,0.687755
249026,Brünste,Brünste,,brünste,LEX_A,LEX_A,k9EqmmVUaG,<NA>,,Deklinierte_Form,1.677223
306877,Körpermasse_1,Körpermasse,,körpermasse,LEX_A,LEX_A,bIf1qblVpr,<NA>,,Deklinierte_Form,0.038776
311771,angesprochenen,angesprochenen,,angesprochenen,LEX_A,LEX_A,SR4oes70ev,<NA>,,Deklinierte_Form,5.850183
376406,verrussende_1,verrussende,,verrussende,LEX_A,LEX_A,z2L4apgJia,<NA>,,Deklinierte_Form,0.142857
644838,Fachleute,Fachleute,,fachleute,NN,NN,fQsLHPCpnX,NOM,N,Deklinierte_Form Substantiv,8.773447


time: 325 ms


In [36]:
# lemma von anderen Datensätzen abschreiben
df = pak.update_col( df_to = df, 
                     df_from = wiktionary_nolemma, 
                     on = ['lex','tag'],
                     col = 'lemma',
                     col_score = 'score'
                    )

update_col: col_score score applied, 15135 records less!
update_col: 13 cells written into existing column
time: 3.83 s


In [37]:
# manuelle Korrekturen
mask = df.lex.isin(['des','dem']) 
df.loc[mask,'lemma'] = 'der'

time: 31.9 ms


In [38]:
df['lemma_lower'] = df.lemma.str.lower()

time: 38.2 ms


In [39]:
search_str(df,suche_debug)

,nolemma_id,lex,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,score


time: 74.9 ms


In [40]:
mask = df.lemma != ''
df[mask]

,nolemma_id,lex,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,score
120,des,des,der,der,LEX_A,LEX_A,VcEqMU7kd0,<NA>,,Deklinierte_Form Artikel,4.706456
121,dem,dem,der,der,LEX_A,LEX_A,DTP9HljRXI,<NA>,,Deklinierte_Form Artikel,4.933009
122,eine,eine,ein,ein,LEX_A,LEX_A,SSmKoBiIl6,<NA>,,Deklinierte_Form Artikel,5.035323
126,einer,einer,ein,ein,LEX_A,LEX_A,vymWo0djqE,<NA>,,Deklinierte_Form Artikel,6.456760
129,einem,einem,ein,ein,LEX_A,LEX_A,hQKwDHLsnN,<NA>,,Deklinierte_Form Artikel,4.180268
10665,heißet_1,heißet,heißen,heißen,LEX_V,LEX_V,VlfISrygIt,<NA>,,Konjugierte_Form Verb,0.965306
46515,Dummies_1,Dummies,Dummie,dummie,LEX_A,LEX_A,5RSrO59nw8,<NA>,,Deklinierte_Form,0.118367
131013,Thors_1,Thors,Thor,thor,LEX_A,LEX_A,ebkM4tNjw0,<NA>,,Deklinierte_Form,0.055102
151298,übergösse_1,übergösse,übergießen,übergießen,LEX_V,LEX_V,x2Uj0lvBjq,<NA>,,Konjugierte_Form Verb,0.687755
306887,Körpermassen_1,Körpermassen,Körpermasse,körpermasse,LEX_A,LEX_A,Jm3Mlvi8Mp,<NA>,,Deklinierte_Form,0.038776


time: 58.9 ms


In [41]:
# wieder anfügen
mask = df.lemma != ''
wiktionary_nolemma = pak.add_rows(wiktionary_nolemma,df[mask])

15 rows added, now a total of 644810
time: 353 ms


In [42]:
# suche_debug 
search_str(wiktionary_nolemma, suche_debug)

,nolemma_id,lex,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,score
63700,gefriergetrocknetem,gefriergetrocknetem,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,qfxGEFcaEM,<NA>,,Deklinierte_Form,0.822449
63701,gefriergetrockneten,gefriergetrockneten,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,cSrtFUpYCE,<NA>,,Deklinierte_Form,2.253350
63702,gefriergetrockneter,gefriergetrockneter,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,cJeBRmnkEZ,<NA>,,Deklinierte_Form,1.298417
63703,gefriergetrocknetes,gefriergetrocknetes,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,ztSD6oEkot,<NA>,,Deklinierte_Form,1.166870
63704,gefriergetrocknete,gefriergetrocknete,gefriergetrocknet,gefriergetrocknet,LEX_A,LEX_A,UYh5zwEhfx,<NA>,,Deklinierte_Form,1.848965
178132,missrietet,missrietet,missraten,missraten,LEX_V,LEX_V,y9ZTeYS8SU,<NA>,,Konjugierte_Form Verb,0.963265
178133,missrieten,missrieten,missraten,missraten,LEX_V,LEX_V,VhpO1YDYiF,<NA>,,Konjugierte_Form Verb,1.144945
178134,missrietst,missrietst,missraten,missraten,LEX_V,LEX_V,5Jms1aP5H9,<NA>,,Konjugierte_Form Verb,0.812245
178135,missrietest,missrietest,missraten,missraten,LEX_V,LEX_V,N0lyUCwTOa,<NA>,,Konjugierte_Form Verb,0.993878
178136,missriete,missriete,missraten,missraten,LEX_V,LEX_V,p6ksoWbv4K,<NA>,,Konjugierte_Form Verb,0.810204


time: 511 ms


In [43]:
# lemma_tagZZ
wiktionary_nolemma['lemma_tagZZ'] = wiktionary_nolemma.tagZZ
wiktionary_nolemma = pak.move_cols(wiktionary_nolemma,'lemma_tagZZ','lemma_lower')

time: 52.5 ms


In [44]:
# lemma_tagZZ = 'N' schreiben 
mask1 =  wiktionary_nolemma.tag.isin(['LEX_A','NN','NE'])
mask2a = wiktionary_nolemma.member.str.contains('Deklinierte_Form')
mask2b = wiktionary_nolemma.member.str.contains('Substantiv')
mask2c = wiktionary_nolemma.member.str.contains('adjektivische_Deklination')
mask3 =  wiktionary_nolemma.lemma.str[0].str.isupper()
mask4 =  wiktionary_nolemma.lemma_tagZZ == ''
mask = mask1  &  (mask2a | mask2b | mask2c)  &  mask3  &  mask4
check_mask(wiktionary_nolemma,mask, 175735)
#wiktionary_nolemma[mask] 
wiktionary_nolemma.loc[mask,'lemma_tagZZ'] = 'N'

check_mask:  175707 rows
time: 1.09 s


In [45]:
# lemma_tagZZ = 'V' schreiben
mask1 =  wiktionary_nolemma.tag == 'LEX_V'
mask2a = wiktionary_nolemma.member.str.contains('Verb') 
mask2b = wiktionary_nolemma.member.str.contains('Konjugierte_Form')
mask2c = wiktionary_nolemma.member.str.contains('Erweiterter_Infinitiv') 
mask3a = wiktionary_nolemma.lemma.str[0].str.islower()
mask3b = wiktionary_nolemma.lemma.str.contains(' ')
mask4 =  wiktionary_nolemma.lemma_tagZZ == ''
mask5 =  wiktionary_nolemma.lemma.str.endswith(verbendungen) # definiert in Steuertabellen
mask = mask1  &  (mask2a | mask2b | mask2c)  &  (mask3a | mask3b)  &  mask4  &  mask5
check_mask(wiktionary_nolemma,mask, 208898)
#wiktionary_nolemma[mask] 
wiktionary_nolemma.loc[mask,'lemma_tagZZ'] = 'V'

check_mask:  208878 rows
time: 1.53 s


In [46]:
# lemma_tagZZ = 'V' schreiben
mask1 =  wiktionary_nolemma.tag == 'LEX_A'
mask2 =  wiktionary_nolemma.member.str.contains('Dekliniertes_Gerundivum')
mask3a = wiktionary_nolemma.lemma.str[0].str.islower()
mask3b = wiktionary_nolemma.lemma.str.contains(' ')
mask4 =  wiktionary_nolemma.lemma_tagZZ == ''
mask5 =  wiktionary_nolemma.lemma.str.endswith(verbendungen) # definiert in Steuertabellen
mask = mask1  &  mask2  &  (mask3a | mask3b)  &  mask4  &  mask5
check_mask(wiktionary_nolemma,mask, 16065)
#wiktionary_nolemma[mask] 
wiktionary_nolemma.loc[mask,'lemma_tagZZ'] = 'V'

check_mask:  16065 rows
time: 988 ms


In [47]:
# lemma_tagZZ = 'A' schreiben
mask1 =  wiktionary_nolemma.tag == 'LEX_A'
mask2a = wiktionary_nolemma.member.str.contains('Deklinierte_Form')  &  ~wiktionary_nolemma.member.str.contains('Artikel')  
mask2b = wiktionary_nolemma.member.str.contains('Adjektiv')
mask2c = wiktionary_nolemma.member.str.contains('Komparativ')
mask2d = wiktionary_nolemma.member.str.contains('Superlativ')
mask3 =  wiktionary_nolemma.lemma.str[0].str.islower()
mask4 =  wiktionary_nolemma.lemma_tagZZ == ''
mask = mask1  &  (mask2a | mask2b | mask2c | mask2d)  &  mask3  &  mask4 
check_mask(wiktionary_nolemma,mask,243281)
#wiktionary_nolemma[mask] 
wiktionary_nolemma.loc[mask,'lemma_tagZZ'] = 'A'

check_mask:  243296 rows
time: 1.58 s


In [48]:
# suche_debug 
search_str(wiktionary_nolemma, suche_debug)

,nolemma_id,lex,lemma,lemma_lower,lemma_tagZZ,tag,tag_0,tag_1,tagZ,tagZZ,member,score
63700,gefriergetrocknetem,gefriergetrocknetem,gefriergetrocknet,gefriergetrocknet,A,LEX_A,LEX_A,qfxGEFcaEM,<NA>,,Deklinierte_Form,0.822449
63701,gefriergetrockneten,gefriergetrockneten,gefriergetrocknet,gefriergetrocknet,A,LEX_A,LEX_A,cSrtFUpYCE,<NA>,,Deklinierte_Form,2.253350
63702,gefriergetrockneter,gefriergetrockneter,gefriergetrocknet,gefriergetrocknet,A,LEX_A,LEX_A,cJeBRmnkEZ,<NA>,,Deklinierte_Form,1.298417
63703,gefriergetrocknetes,gefriergetrocknetes,gefriergetrocknet,gefriergetrocknet,A,LEX_A,LEX_A,ztSD6oEkot,<NA>,,Deklinierte_Form,1.166870
63704,gefriergetrocknete,gefriergetrocknete,gefriergetrocknet,gefriergetrocknet,A,LEX_A,LEX_A,UYh5zwEhfx,<NA>,,Deklinierte_Form,1.848965
178132,missrietet,missrietet,missraten,missraten,V,LEX_V,LEX_V,y9ZTeYS8SU,<NA>,,Konjugierte_Form Verb,0.963265
178133,missrieten,missrieten,missraten,missraten,V,LEX_V,LEX_V,VhpO1YDYiF,<NA>,,Konjugierte_Form Verb,1.144945
178134,missrietst,missrietst,missraten,missraten,V,LEX_V,LEX_V,5Jms1aP5H9,<NA>,,Konjugierte_Form Verb,0.812245
178135,missrietest,missrietest,missraten,missraten,V,LEX_V,LEX_V,N0lyUCwTOa,<NA>,,Konjugierte_Form Verb,0.993878
178136,missriete,missriete,missraten,missraten,V,LEX_V,LEX_V,p6ksoWbv4K,<NA>,,Konjugierte_Form Verb,0.810204


time: 580 ms


In [49]:
# bak
wiktionary_nolemma_bak = wiktionary_nolemma.copy()

time: 293 ms


In [50]:
# lemma_id schreiben
wiktionary_nolemma = pak.update_col( df_to =      wiktionary_nolemma, 
                                     df_from =    wiktionary_lemma, 
                                     left_on =    ['lemma','lemma_tagZZ'],
                                     right_on =   ['lemma','tagZZ'],
                                     col =        'lemma_id',
                                     col_score = 'score'
                                            )
wiktionary_nolemma = pak.move_cols(wiktionary_nolemma,['lemma_id','lemma','lemma_lower','lemma_tagZZ'],-1)

update_col: col_score score applied, 4126 records less!
update_col: 640609 cells written into new column
time: 3.57 s


In [51]:
# lemma_id ergänzen
wiktionary_nolemma, mask = pak.update_col( df_to =      wiktionary_nolemma, 
                                           df_from =    wiktionary_lemma, 
                                           on =         'lemma',
                                           col =        'lemma_id',
                                           col_score =  'score',
                                           cond =       'null',
                                           return_mask= True
                                            )
#wiktionary_nolemma.loc[mask,'member'] += ' lemma_id?'

update_col: col_score score applied, 4711 records less!
update_col: 3593 cells written into existing column
time: 3.74 s


In [52]:
#grid(wiktionary_nolemma,mask)

time: 40.9 ms


In [53]:
# translate-Tabelle bauen für lemmas aus wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
translate = pak.group_and_agg( wiktionary_merkmal[mask], 
              [ 'data',    'lemma_id',  'score',   ],
              [ 'group',   'group',     'sum',     ],
              [ '',        '',          'score',   ],        
             )
translate = pak.rank(translate, col_score='score', cols_group='data', on_conflict='first')

mask = translate.data == 'Fachleute'
translate[mask]

532862 rows less, now 575388 rows
rank: 126682 rows less, now 448706 rows


,data,lemma_id,score
96776,Fachleute,Fachmann,54.887939


time: 3.2 s


In [54]:
# lemma_id via lex aus translate ergänzen
wiktionary_nolemma = pak.update_col( df_to =      wiktionary_nolemma, 
                                     df_from =    translate, 
                                     left_on =    'lex',
                                     right_on =   'data',                                    
                                     col =        'lemma_id',
                                     cond =       'null',
                                            )

update_col: 35 cells written into existing column
time: 3.87 s


In [55]:
# lemma_id via lemma aus translate ergänzen
wiktionary_nolemma = pak.update_col( df_to =      wiktionary_nolemma, 
                                     df_from =    translate, 
                                     left_on =    'lemma',
                                     right_on =   'data',                                    
                                     col =        'lemma_id',
                                     cond =       'null',
                                            )

update_col: 467 cells written into existing column
time: 3.76 s


In [56]:
# translate-Tabelle bauen für Deklinierte Formen aus wiktionary_nolemma
# Beispiel: Brandschutzverantwortliche 
#mask = wiktionary_merkmal.is_lex == True
translate = pak.group_and_agg( wiktionary_nolemma, 
              [ 'lex',    'lemma_id',  'score',   ],
              [ 'group',   'group',    'sum',     ],
              [ 'data',    '',         'score',   ],        
             )
translate = pak.rank(translate, col_score='score', cols_group='data', on_conflict='first')

mask = translate.data.isin(['Brandschutzverantwortliche','Brandschutzverantwortlicher','Brandschutzverantwortlichen'])
translate[mask]

13826 rows less, now 630984 rows
rank: 9662 rows less, now 621322 rows


,data,lemma_id,score
23080,Brandschutzverantwortlichen,Brandschutzverantwortlicher,3.356882
23078,Brandschutzverantwortliche,Brandschutzverantwortlicher,1.032887
23081,Brandschutzverantwortlicher,Brandschutzverantwortlicher,1.006090


time: 2.27 s


In [57]:
# lemma_id via lex aus translate ergänzen
wiktionary_nolemma = pak.update_col( df_to =      wiktionary_nolemma, 
                                     df_from =    translate, 
                                     left_on =    'lex',
                                     right_on =   'data',                                    
                                     col =        'lemma_id',
                                     cond =       'null',
                                            )

update_col: 45 cells written into existing column
time: 3.89 s


In [58]:
# lemma_id via lemma aus translate ergänzen
wiktionary_nolemma = pak.update_col( df_to =      wiktionary_nolemma, 
                                     df_from =    translate, 
                                     left_on =    'lemma',
                                     right_on =   'data',                                    
                                     col =        'lemma_id',
                                     cond =       'null',
                                            )

update_col: 48 cells written into existing column
time: 3.69 s


In [59]:
# lemma_tagZZ zurückschreiben
wiktionary_nolemma = pak.update_col( df_to =      wiktionary_nolemma, 
                                     df_from =    wiktionary_lemma, 
                                     on =         'lemma_id',
                                     col =        'tagZZ',
                                     col_rename = 'lemma_tagZZ',                                    
                                     col_score =  'score',
                                            )

update_col: col_score score applied, but it was pointless!
update_col: 644790 cells written into existing column
time: 4.06 s


In [60]:
# Kontrolle (ändere mask auf A, V oder N und kontrolliere die lemmas)
mask = wiktionary_nolemma.lemma_tagZZ == 'A'
a = pak.analyse_freqs(wiktionary_nolemma[mask], ['member','tag','lemma'])
grid(a)

6 rows


time: 1.41 s


In [61]:
# Die meisten lemma_id konnten gefunden werden
mask = wiktionary_nolemma.lemma_id.isnull()
check_mask(wiktionary_nolemma,mask,0,20)
grid(wiktionary_nolemma,mask)

check_mask:  13 rows
13 rows out of 644810


time: 257 ms


In [62]:
#mask1 = wiktionary_nolemma.lemma_id.isnull()
#mask2 = wiktionary_nolemma.member.str.contains('Erweiterter_Infinitiv')
#mask = mask1  &  mask2
#grid(wiktionary_nolemma,mask)

time: 72.1 ms


In [63]:
# suche_debug 
search_str(wiktionary_nolemma, suche_debug)

,nolemma_id,lex,tag,tag_0,tag_1,tagZ,tagZZ,member,score,lemma_id,lemma,lemma_lower,lemma_tagZZ
63700,gefriergetrocknetem,gefriergetrocknetem,LEX_A,LEX_A,qfxGEFcaEM,<NA>,,Deklinierte_Form,0.822449,gefriergetrocknet,gefriergetrocknet,gefriergetrocknet,A
63701,gefriergetrockneten,gefriergetrockneten,LEX_A,LEX_A,cSrtFUpYCE,<NA>,,Deklinierte_Form,2.253350,gefriergetrocknet,gefriergetrocknet,gefriergetrocknet,A
63702,gefriergetrockneter,gefriergetrockneter,LEX_A,LEX_A,cJeBRmnkEZ,<NA>,,Deklinierte_Form,1.298417,gefriergetrocknet,gefriergetrocknet,gefriergetrocknet,A
63703,gefriergetrocknetes,gefriergetrocknetes,LEX_A,LEX_A,ztSD6oEkot,<NA>,,Deklinierte_Form,1.166870,gefriergetrocknet,gefriergetrocknet,gefriergetrocknet,A
63704,gefriergetrocknete,gefriergetrocknete,LEX_A,LEX_A,UYh5zwEhfx,<NA>,,Deklinierte_Form,1.848965,gefriergetrocknet,gefriergetrocknet,gefriergetrocknet,A
178132,missrietet,missrietet,LEX_V,LEX_V,y9ZTeYS8SU,<NA>,,Konjugierte_Form Verb,0.963265,missraten,missraten,missraten,V
178133,missrieten,missrieten,LEX_V,LEX_V,VhpO1YDYiF,<NA>,,Konjugierte_Form Verb,1.144945,missraten,missraten,missraten,V
178134,missrietst,missrietst,LEX_V,LEX_V,5Jms1aP5H9,<NA>,,Konjugierte_Form Verb,0.812245,missraten,missraten,missraten,V
178135,missrietest,missrietest,LEX_V,LEX_V,N0lyUCwTOa,<NA>,,Konjugierte_Form Verb,0.993878,missraten,missraten,missraten,V
178136,missriete,missriete,LEX_V,LEX_V,p6ksoWbv4K,<NA>,,Konjugierte_Form Verb,0.810204,missraten,missraten,missraten,V


time: 957 ms


In [64]:
# trash erzeugen  
wiktionary_nolemma_trash = wiktionary_nolemma.head(0)
wiktionary_nolemma_trash['msg'] = None

time: 38.6 ms


In [65]:
# Schrott löschen
mask = wiktionary_nolemma.lemma_id.isnull()
check_mask( wiktionary_nolemma, mask, 0, 20)
msg = 'Schrott_lemma_id'
wiktionary_nolemma, wiktionary_nolemma_trash = pak.move_rows(wiktionary_nolemma, wiktionary_nolemma_trash, mask, msg)
grid(wiktionary_nolemma_trash, msg)

check_mask:  13 rows
Move 13 rows from 644810
13 rows


time: 983 ms


In [66]:
# suche_debug wiktionary_nolemma
mask = wiktionary_nolemma_trash.lemma.isin(suche_debug)
wiktionary_nolemma_trash[mask]

,nolemma_id,lex,tag,tag_0,tag_1,tagZ,tagZZ,member,score,lemma_id,lemma,lemma_lower,lemma_tagZZ,msg


time: 90 ms


In [67]:
grid(wiktionary_nolemma, pak.sample)

17 rows out of 644797


time: 5.76 s


In [68]:
# fillna
wiktionary_nolemma['tagZ'] = wiktionary_nolemma.tagZ.fillna('')

time: 146 ms


### Trash erzeugen

In [69]:
# trash erzeugen     
wiktionary_lemma_trash   = wiktionary_lemma.head(0)
wiktionary_lemma_trash['msg'] = None

# ist bereits erzeugt
#wiktionary_nolemma_trash = wiktionary_nolemma.head(0)
#wiktionary_nolemma_trash['msg'] = None

wiktionary_merkmal_trash = wiktionary_merkmal.head(0)
wiktionary_merkmal_trash['msg'] = None

wiktionary_merkmal_text_trash = wiktionary_merkmal_text.head(0)
wiktionary_merkmal_text_trash['msg'] = None

time: 75.4 ms


### wiktionary_merkmal
* Es wird versucht, möglichst jedes `data` in wiktionary_lemma oder wiktionary_nolemma zu finden und diese id als `data_id` zu schreiben.
* Zuerst werden die data_id spezieller Fälle geschieben, anschließend alle anderen data_id.

In [70]:
# fillna
wiktionary_merkmal['num']     = wiktionary_merkmal.num.fillna('')
wiktionary_merkmal['data_id'] = wiktionary_merkmal.data_id.fillna('')
wiktionary_merkmal['score']   = wiktionary_merkmal.score.fillna(0)

time: 886 ms


In [71]:
# Hochkomma in data streichen
mask1 = wiktionary_merkmal.data.str.contains("'")
mask2 = wiktionary_merkmal.merkmal.isin(['def'])
mask = mask1  &  mask2
wiktionary_merkmal.loc[mask,'data'] = pak.remove_str(wiktionary_merkmal[mask].data,"'")

time: 1.9 s


In [72]:
# Dups entfernen
mask = wiktionary_merkmal.duplicated(subset=['lemma_id','merkmal','data','num','meta'])
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, 'dup')

Move 13247 rows from 4987597
time: 8.19 s


In [73]:
# lemma_tag schreiben
wiktionary_merkmal = pak.update_col( df_to =      wiktionary_merkmal, 
                                     df_from =    wiktionary_lemma, 
                                     on =         'lemma_id',
                                     col =        'tag',
                                     col_rename = 'lemma_tag',
                                            )
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'lemma_tag','lemma_id')
wiktionary_merkmal['lemma_tag'] = wiktionary_merkmal.lemma_tag.fillna('')

update_col: 2669660 cells written into new column
time: 13.8 s


In [74]:
# Vergebene data_id eindeutig?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

time: 321 ms


In [75]:
# Nur Adjektive / Ptz1: data_id schreiben aus wiktionary_lemma
mask_L = wiktionary_lemma.tagZ == 'ADJ'
mask_M = wiktionary_merkmal.merkmal.str.contains('Ptz1')  |  wiktionary_merkmal.merkmal.str.contains('Ptz2')

wiktionary_merkmal.loc[mask_M] = pak.update_col( df_to =      wiktionary_merkmal[mask_M], 
                                                 df_from =    wiktionary_lemma[mask_L], 
                                                 left_on =    'data',
                                                 right_on =   'lemma',
                                                 col =        'lemma_id',
                                                 col_rename = 'data_id',
                                                 col_score =  'score',
                                                 cond =       'null',                                       
                                            )

update_col: col_score score applied, 1134 records less!
update_col: 811 cells written into existing column
time: 4.65 s


In [76]:
# Vergebene data_id eindeutig?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

time: 329 ms


In [77]:
#mask = wiktionary_merkmal.data == 'anscheinend'
#wiktionary_merkmal[mask]

time: 37.1 ms


In [78]:
# data_id schreiben aus wiktionary_lemma, tag muss übereinstimmen
wiktionary_merkmal = pak.update_col( df_to =      wiktionary_merkmal, 
                                     df_from =    wiktionary_lemma, 
                                     left_on =    ['data','lemma_tag'],
                                     right_on =   ['lemma','tag'],
                                     col =        'lemma_id',
                                     col_rename = 'data_id',
                                     col_score =  'score',
                                     cond =       'null',                                    
                                            )

update_col: col_score score applied, 3019 records less!
update_col: 984655 cells written into existing column
time: 18.3 s


In [79]:
# Vergebene data_id eindeutig?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

time: 1.92 s


In [80]:
# data_id ergänzen aus wiktionary_lemma, alle sonstigen
wiktionary_merkmal = pak.update_col( df_to =      wiktionary_merkmal, 
                                     df_from =    wiktionary_lemma, 
                                     left_on =    'data',
                                     right_on =   'lemma',
                                     col =        'lemma_id',
                                     col_rename = 'data_id',
                                     col_score =  'score',
                                     cond =       'null',
                                            )

update_col: col_score score applied, 4711 records less!
update_col: 988475 cells written into existing column
time: 17.4 s


In [81]:
# Vergebene data_id eindeutig?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

time: 2.79 s


In [82]:
# data_id ergänzen aus wiktionary_nolemma
wiktionary_merkmal = pak.update_col( df_to =      wiktionary_merkmal, 
                                     df_from =    wiktionary_nolemma, 
                                     left_on =    'data',
                                     right_on =   'lex',
                                     col =        'nolemma_id',
                                     col_rename = 'data_id',
                                     col_score =  'score',
                                     cond =       'null',
                                            )

update_col: col_score score applied, 23427 records less!
update_col: 605081 cells written into existing column
time: 19.9 s


In [83]:
# Vergebene data_id eindeutig?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

time: 4.11 s


In [84]:
# Backup ziehen
#wiktionary_merkmal_bak = wiktionary_merkmal.copy()
#wiktionary_merkmal = wiktionary_merkmal_bak.copy()

time: 37.4 ms


In [85]:
# data_id ist Verweis auf das eigene lemma
mask1a = wiktionary_merkmal.merkmal.str.endswith(('Sg','Pl','Sg m','Sg f','Sg n', 'Pl m', 'Pl f', 'Pl n'))
mask1b = wiktionary_merkmal.merkmal.str.endswith((' ich',' du',' ersiees',' wir',' ihr',' sie'))
mask1c = wiktionary_merkmal.merkmal.isin(['VAFIN','lex','weib','gm','klein','abk','Ptz2','männ','alt','alt_1','alt_2',
                                          'alt_3','lexAlt','alt_abk','alt_klein','alt_weib','alt_männ','Gen','VVIZU','Superlativ','Komparativ','VMFIN'])
mask2 =  wiktionary_merkmal.data_id == ''
mask3 =  wiktionary_merkmal.is_lex == True
mask = (mask1a | mask1b | mask1c)  &  mask2  &  mask3  
check_mask(wiktionary_merkmal,mask,55025)

wiktionary_merkmal.loc[mask,'data_id'] = wiktionary_merkmal[mask].lemma_id + '~' + wiktionary_merkmal[mask].data

check_mask:  55067 rows
time: 4.28 s


In [86]:
#grid(wiktionary_merkmal,mask)

time: 38.1 ms


In [87]:
# Vergebene data_id eindeutig?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

time: 4.36 s


In [88]:
# Artefakte
artefakte = ['Nebensatzkonjugation','Hauptsatzkonjugation']
mask = wiktionary_merkmal.data.isin(artefakte)
check_mask(wiktionary_merkmal,mask,60)
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal,wiktionary_merkmal_trash, mask, 'artefakte')

check_mask:  61 rows
Move 61 rows from 4974350
time: 3.21 s


In [89]:
# gm Merkmale
mask1 =  wiktionary_merkmal.merkmal == 'gm'
mask2 =  wiktionary_merkmal.data_id == ''
mask3 = ~wiktionary_merkmal.data.str.contains('\(')
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,114)
#grid(wiktionary_merkmal,mask)
wiktionary_merkmal.loc[mask,'data_id'] = wiktionary_merkmal[mask].lemma_id + '~' + wiktionary_merkmal[mask].data

check_mask:  108 rows
time: 2.74 s


In [90]:
# Vergebene data_id eindeutig?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

time: 4.5 s


In [91]:
# data_tag
wiktionary_merkmal['data_tag'] = ''

time: 58.7 ms


In [92]:
# Spalten sortieren
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,['lemma_id','merkmal','data_id','data','data_tag','lemma_tag','num','meta','score'])
wiktionary_merkmal.head(1)

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
0,Hallo,syn,Gejohle,Gejohle,,NN,1,,7.282582,False


time: 127 ms


In [93]:
# Prüfen: Fast alle Lexeme haben eine data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask_noid = mask1  &  mask2
check_mask(wiktionary_merkmal,mask_noid,0,6)

check_mask:  6 rows
time: 578 ms


In [94]:
# Fehlende data_id finden
mask1 = wiktionary_merkmal.data.isin(wiktionary_merkmal[mask_noid].data)
mask2 = wiktionary_merkmal.data_id != ''
mask = mask1  &  mask2
wiktionary_merkmal[mask]

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
1642057,zu_viele,alt,zu_viele~zuviele,zuviele,,,,,1.412911,True


time: 815 ms


In [95]:
# data_id ergänzen aus wiktionary_merkmal selbst
wiktionary_merkmal.loc[mask_noid] = pak.update_col( df_to =      wiktionary_merkmal[mask_noid], 
                                                    df_from =    wiktionary_merkmal[mask],
                                                    on =         'data',
                                                    col =        'data_id',
                                            )

update_col: 1 cells written into existing column
time: 1.97 s


In [96]:
# Löschen: Lexeme ohne data_id
mask1  = wiktionary_merkmal.data_id == ''
mask2A = wiktionary_merkmal.is_lex == True
mask2B = wiktionary_merkmal.merkmal == 'gm'
mask_noid = mask1  &  (mask2A | mask2B)
check_mask(wiktionary_merkmal,mask_noid,0,15)

msg = 'no data_id'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal,wiktionary_merkmal_trash, mask_noid, msg)
grid(wiktionary_merkmal_trash,msg)

check_mask:  12 rows
Move 12 rows from 4974289
12 rows out of 13320


time: 3.89 s


### wiktionary_merkmal_wertlos
* nimmt alle Datensätze aus wiktionary_merkmal, deren data_id leer geblieben ist und die auch sonst keinen Nutzen haben

In [97]:
# Merkmale, die ohne data_id auskommen
print(merkmale_ohne_data_id)

['ipa', 'syllables', 'rhymes', 'Genus', 'up']
time: 45.5 ms


In [98]:
# Datensätze ohne data_id
mask1 =  wiktionary_merkmal.data_id == ''
mask2 = ~wiktionary_merkmal.merkmal.isin(merkmale_ohne_data_id)
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,323911) 
#grid(wiktionary_merkmal[mask]) 

check_mask:  323913 rows
time: 993 ms


In [99]:
# Reste: data_id ist null geblieben
a = pak.analyse_freqs(wiktionary_merkmal[mask],'merkmal')
grid(a)

14 rows


time: 490 ms


In [100]:
# einzelne Merkmale betrachten
mask3 = wiktionary_merkmal.merkmal == 'def'
grid(wiktionary_merkmal[mask & mask3]) 

28007 rows


time: 999 ms


In [101]:
# Datensätze in wiktionary_merkmal_wertlos verschieben
wiktionary_merkmal, wiktionary_merkmal_wertlos = pak.move_rows(wiktionary_merkmal,mask, msg=None)

Move 323913 rows from 4974277
time: 4 s


In [102]:
# drop_cols
wiktionary_merkmal_wertlos = pak.drop_cols(wiktionary_merkmal_wertlos,['data_id'])

time: 152 ms


In [103]:
# Prüfen, ob da unerlaubte Merkmale drin sind
mask1 = ~wiktionary_merkmal_wertlos.merkmal.isin(merkmale_in_wiktionary_merkmal_wertlos)
mask2 = ~wiktionary_merkmal_wertlos.meta.str.contains('manuell')
mask = mask1  &  mask2
check_mask(wiktionary_merkmal_wertlos,mask,0)
#grid(wiktionary_merkmal_wertlos,mask)

check_mask:  0 rows
time: 164 ms


In [104]:
# Häufig vorkommende Worte
a = pak.analyse_freqs(wiktionary_merkmal_wertlos,'data')
grid(a.head(20))

20 rows


time: 687 ms


In [105]:
#mask = wiktionary_merkmal_wertlos.data == 'Essen und Trinken/Küchenkräuter und Gewürze'
#wiktionary_merkmal_wertlos[mask]

time: 67.7 ms


In [106]:
#pak.analyse_cols(wiktionary_merkmal_wertlos)

time: 92.7 ms


In [107]:
mask = wiktionary_merkmal.data_id == ''
wiktionary_merkmal[mask]

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
8,Subfamilia,up,,Familia,,NN,1,,5.028015,False
104,Dezember,up,,Zeit und Kalender,,NN,1,,23.587090,False
227,März,up,,Zeit und Kalender,,NN,1,auch,20.813643,False
258,April,up,,Zeit und Kalender,,NN,1,auch,28.487211,False
288,Mai,up,,Zeit und Kalender,,NN,1,auch,25.984165,False
...,...,...,...,...,...,...,...,...,...,...
4987591,rüffel,syllables,,rüf⇀fel,,,,,1.153471,False
4987592,anmaulte,syllables,,an⇀maul⇀te,,,,,1.140073,False
4987594,maultest_an,syllables,,maul⇀test⇀an,,,,,0.742857,False
4987595,prekärster,syllables,,pre⇀kärs⇀ter,,,,,1.244823,False


time: 1.22 s


### wiktionary_lemma Spacy
* doc_lemma: Welches lemma ermittelt Spacy?
* doc_lemma: Welchen tag ermittelt Spacy?
* doc_len

In [108]:
def analyse_spacy_wl(zeile):
    doc = nlp(zeile.lemma)
    zeile['doc_len']   = len(doc)  
    if zeile['doc_len']  < 1:
        return zeile    
    
    zeile['doc_lemma'] = ' '.join([token.lemma_ for token in doc])
    zeile['doc_tag']   = ' '.join([token.tag_   for token in doc])    
    
    return zeile

time: 36.7 ms


In [109]:
# apply analyse_spacy_wl 
if gründlich:
    wiktionary_lemma['doc_len']   = None
    wiktionary_lemma['doc_lemma'] = None
    wiktionary_lemma['doc_tag']   = None 

    wiktionary_lemma = wiktionary_lemma.swifter.apply( analyse_spacy_wl, axis=1 )
    wiktionary_lemma = pak.change_datatype(wiktionary_lemma, verbose=False)  

time: 9min 28s


In [110]:
# suche_debug wiktionary_lemma
if gründlich:
    spalten = ['lemma_id','lemma','page_part','tag','doc_len','doc_lemma','doc_tag']
    search_str(wiktionary_lemma[spalten], suche_debug)

time: 150 ms


## Prüfen NaN

In [111]:
assert not pak.any_nan(wiktionary_lemma)
#pak.nnan(wiktionary_lemma)

time: 122 ms


In [112]:
assert not pak.any_nan(wiktionary_nolemma)
#pak.nnan(wiktionary_nolemma)

time: 411 ms


In [113]:
assert not pak.any_nan(wiktionary_merkmal)
#pak.nnan(wiktionary_merkmal)

time: 2.09 s


In [114]:
assert not pak.any_nan(wiktionary_merkmal_wertlos)
#pak.nnan(wiktionary_merkmal_wertlos)

time: 167 ms


In [115]:
# Prüfen: is_unique
assert wiktionary_lemma.lemma_id.is_unique

time: 81.1 ms


In [116]:
# Prüfen: is_unique
assert wiktionary_nolemma.nolemma_id.is_unique

time: 198 ms


## Prüfen lex_test

In [117]:
# lex_test, lemma_test
from x32_Funktionen import *
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=2 )
#grid(lemma_test)

74 rows loaded
Delete 17 rows from 74
time: 1.61 s


In [118]:
# lemma_test
from x32_Funktionen import *
lemma_test = check_lemma_test(lemma_test, wiktionary_lemma)
#lemma_test

time: 3.38 s


In [119]:
# Fehler
maskA = lemma_test.lemma_id == ''
maskB = lemma_test.check_tag == ''
mask = maskA | maskB
check_mask(lemma_test,mask)

probleme = list(lemma_test[mask].lemma)
lemma_test[mask]

check_mask:  0 rows


,level,lemma_id,lemma,tag_soll,tag_ist,member_soll,check_tag,member_ist,check_member


time: 45.3 ms


In [120]:
# Sicherstellen: Keine Fehler
check_mask(lemma_test,mask,0)

check_mask:  0 rows
time: 46.5 ms


## suche_debug

### wiktionary_lemma

In [121]:
mask = wiktionary_lemma.lemma.isin(suche_debug) 
wiktionary_lemma[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,doc_len,doc_lemma,doc_tag
2054,gefriergetrocknet,273380,0,gefriergetrocknet,gefriergetrocknet,False,1,ADJA,ADJA,s6ixiUi9BM,ADJ,A,Ptz2 noSteig manuell,gefriertrocknen,4.307552,1,friergetrocknen,VVPP
2055,gefriertrocknen,1054310,0,gefriertrocknen,gefriertrocknen,False,1,VVFIN,VVFIN,yJnWihTkXL,VER,V,manuell,,14.872655,1,gefriertrockn,VVPP
2194,missraten,242150,0,missraten,missraten,False,1,VVFIN,VVFIN,DXXh04S9JZ,VER,V,manuell,,4.922594,1,Missrat,VVFIN
140462,missraten_MADJ,-1,-1,missraten,missraten,False,1,ADJA,ADJA,dJ48jJBIod,ADJ,A,Ptz2 manuell,missraten,0.500000,1,Missrat,VVFIN


time: 70.6 ms


### wiktionary_nolemma

In [122]:
# wiktionary_nolemma suche_debug
grid(search_str(wiktionary_nolemma, suche_debug))

35 rows


time: 735 ms


### wiktionary_merkmal

In [123]:
mask = wiktionary_merkmal.lemma_id == suche_debug[0]
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

7 rows


time: 647 ms


In [124]:
mask = wiktionary_merkmal.data.isin(suche_debug) 
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

55 rows


time: 371 ms


In [125]:
mask = wiktionary_merkmal.data.isin(suche_debug) 
a = wiktionary_merkmal[mask]
grid(a)

55 rows


time: 461 ms


In [126]:
# Genügend Datensätze?
if version == '_all':
    assert wiktionary_merkmal.shape[0] > 3320 * 1000

time: 43.7 ms


### wiktionary_merkmal_wertlos

In [127]:
mask = wiktionary_merkmal_wertlos.lemma_id == suche_debug[0]
a = wiktionary_merkmal_wertlos[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

No rows
time: 162 ms


In [128]:
grid(wiktionary_merkmal_wertlos,pak.sample)

15 rows out of 323913


time: 3.1 s


## Partizipien prüfen

In [129]:
suche_L = 'missraten' 
suche_P = 'missraten'  

#suche_L = 'gefriertrocknen' 
#suche_P = 'gefriergetrocknet'  

time: 49.1 ms


In [130]:
# das Verb in wiktionary_lemma
mask1 = wiktionary_lemma.lemma == suche_L
mask2 = wiktionary_lemma.tagZZ == 'V'
mask = mask1  &  mask2

suche_L_id = wiktionary_lemma[mask].iloc[0].lemma_id
wiktionary_lemma[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,doc_len,doc_lemma,doc_tag
2194,missraten,242150,0,missraten,missraten,False,1,VVFIN,VVFIN,DXXh04S9JZ,VER,V,manuell,,4.922594,1,Missrat,VVFIN


time: 110 ms


In [131]:
# das Adjektiv in wiktionary_lemma
mask1 = wiktionary_lemma.lemma == suche_P
mask2 = wiktionary_lemma.tagZZ == 'A'
mask3 = wiktionary_lemma.member.str.contains('Ptz')
mask4 = wiktionary_lemma.data_id == suche_L_id
mask = mask1  &  mask2  &  mask3  &  mask4

suche_P_id = wiktionary_lemma[mask].iloc[0].lemma_id
wiktionary_lemma[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,doc_len,doc_lemma,doc_tag
140462,missraten_MADJ,-1,-1,missraten,missraten,False,1,ADJA,ADJA,dJ48jJBIod,ADJ,A,Ptz2 manuell,missraten,0.5,1,Missrat,VVFIN


time: 189 ms


In [132]:
# Das Ptz-Merkmal in wiktionary_lemma
mask1 = wiktionary_merkmal.data == suche_P
mask2 = wiktionary_merkmal.merkmal.str.startswith('Ptz')
mask3 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask4 = wiktionary_merkmal.lemma_id == suche_L_id   # zeigt aufs Verb
mask5 = wiktionary_merkmal.data_id  == suche_P_id   # zeigt aufs Adjektiv
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
wiktionary_merkmal[mask]

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
3325725,missraten,Ptz2,missraten_MADJ,missraten,,VVFIN,,,4.472594,True


time: 4.22 s


## Speichern

In [133]:
assert gründlich

time: 34.3 ms


In [134]:
# Speichern  
pak.dump_pickle(wiktionary_lemma,            wiktionary_lemma_filename_save)   
pak.dump_pickle(wiktionary_nolemma,          wiktionary_nolemma_filename_save)  
pak.dump_pickle(wiktionary_merkmal,          wiktionary_merkmal_filename_save) 
pak.dump_pickle(wiktionary_merkmal_text,     wiktionary_merkmal_text_filename_save)  
pak.dump_pickle(wiktionary_merkmal_wertlos,  wiktionary_merkmal_wertlos_filename_save)  
os.utime(verzeichnis_save)

time: 47.9 s


In [ ]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av {verzeichnis_save} /content/drive/MyDrive